# Oznaczenia:

### Nazwy zmiennych:
* **State** - tablica stanu
* **RoundKey** - podklucz rundy
* **S_box** - skrzynka podstawieniowa
* **Inv_S_box** - odwrotna skrzynka podstawieniowa
* **R_con** - stała rundy
* **Nb** - rozmiar bloku
* **Nr** - liczba rund
* **Nk** - Liczba 32-bitowych słów stanowiących klucz do szyfrowania. Standard przewiduje, że Nk = 4, 6 albo 8 (Sekcja 6.3)
* **temp** - zmienna pomocnicza

### Operatory:
* **^^** - bitowe XOR
* **&** - bitowe AND
* **>>** - Przesuwa bity liczby w prawo o określoną liczbę bitów
* **<<** - Przesuwa bity liczby w lewo o określoną liczbę bitów

### Dodatkowa informacja
* Bajty są zapisywane w postaci dwóch liczb szesnastkowych. Np. 0xf5 - liczba szesnastkowa, wtedy bajt zapisujemy jako 'f5'

# Dane wstępne:

In [1]:
# Importujemy dodatkową funkcję, która wyświetla macierzy w sposób czytelny 
from pprint import pprint as pp


# S-box macierz:
S_box=[
    '63','7c','77','7b','f2','6b','6f','c5','30','01','67','2b','fe','d7','ab','76',
    'ca','82','c9','7d','fa','59','47','f0','ad','d4','a2','af','9c','a4','72','c0',
    'b7','fd','93','26','36','3f','f7','cc','34','a5','e5','f1','71','d8','31','15',
    '04','c7','23','c3','18','96','05','9a','07','12','80','e2','eb','27','b2','75',
    '09','83','2c','1a','1b','6e','5a','a0','52','3b','d6','b3','29','e3','2f','84',
    '53','d1','00','ed','20','fc','b1','5b','6a','cb','be','39','4a','4c','58','cf',
    'd0','ef','aa','fb','43','4d','33','85','45','f9','02','7f','50','3c','9f','a8',
    '51','a3','40','8f','92','9d','38','f5','bc','b6','da','21','10','ff','f3','d2',
    'cd','0c','13','ec','5f','97','44','17','c4','a7','7e','3d','64','5d','19','73',
    '60','81','4f','dc','22','2a','90','88','46','ee','b8','14','de','5e','0b','db',
    'e0','32','3a','0a','49','06','24','5c','c2','d3','ac','62','91','95','e4','79',
    'e7','c8','37','6d','8d','d5','4e','a9','6c','56','f4','ea','65','7a','ae','08',
    'ba','78','25','2e','1c','a6','b4','c6','e8','dd','74','1f','4b','bd','8b','8a',
    '70','3e','b5','66','48','03','f6','0e','61','35','57','b9','86','c1','1d','9e',
    'e1','f8','98','11','69','d9','8e','94','9b','1e','87','e9','ce','55','28','df',
    '8c','a1','89','0d','bf','e6','42','68','41','99','2d','0f','b0','54','bb','16'
]


# Odwrotna S-box macierz:
Inv_S_box = [
    '52','09','6a','d5','30','36','a5','38','bf','40','a3','9e','81','f3','d7','fb',
    '7c','e3','39','82','9b','2f','ff','87','34','8e','43','44','c4','de','e9','cb',
    '54','7b','94','32','a6','c2','23','3d','ee','4c','95','0b','42','fa','c3','4e',
    '08','2e','a1','66','28','d9','24','b2','76','5b','a2','49','6d','8b','d1','25',
    '72','f8','f6','64','86','68','98','16','d4','a4','5c','cc','5d','65','b6','92',
    '6c','70','48','50','fd','ed','b9','da','5e','15','46','57','a7','8d','9d','84',
    '90','d8','ab','00','8c','bc','d3','0a','f7','e4','58','05','b8','b3','45','06',
    'd0','2c','1e','8f','ca','3f','0f','02','c1','af','bd','03','01','13','8a','6b',
    '3a','91','11','41','4f','67','dc','ea','97','f2','cf','ce','f0','b4','e6','73',
    '96','ac','74','22','e7','ad','35','85','e2','f9','37','e8','1c','75','df','6e',
    '47','f1','1a','71','1d','29','c5','89','6f','b7','62','0e','aa','18','be','1b',
    'fc','56','3e','4b','c6','d2','79','20','9a','db','c0','fe','78','cd','5a','f4',
    '1f','dd','a8','33','88','07','c7','31','b1','12','10','59','27','80','ec','5f',
    '60','51','7f','a9','19','b5','4a','0d','2d','e5','7a','9f','93','c9','9c','ef',
    'a0','e0','3b','4d','ae','2a','f5','b0','c8','eb','bb','3c','83','53','99','61',
    '17','2b','04','7e','ba','77','d6','26','e1','69','14','63','55','21','0c','7d'
]


# Stała rundy Rcon:
R_con = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)

# Funkcje pomocnicze:

In [3]:
def transponowac(State):
    """
    Transponuje podaną macierz.
        
        
    WEJŚCIE:
        macierz
        
    WYJŚCIE:
        Transponowana macierz
        
    PRZYKŁAD:
        sage: State = [
                 ['61', '70', '61', '73'],
                 ['73', '77', '6F', '72'],
                 ['64', '63', '69', '70'],
                 ['74', '68', '65', '72']
              ]
        sage: State = transponowac(State)
        sage: pp(State)
           [['61', '73', '64', '74'],
            ['70', '77', '63', '68'],
            ['61', '6F', '69', '65'],
            ['73', '72', '70', '72']]
            
    """
    
    return [[State[j][i] for j in range(len(State))] for i in range(len(State[0]))]



def string_w_wektor_bajtow(tekst):
    """
    Konwertuje napis znaków do wektora, którego elementami są bajty (2 liczby szesnastkowe)
    
    
    WEJŚCIE:
        Tekst znaków
        
    WYJŚCIE:
        Wektor bajtów
        
    PRZYKŁAD:
        sage: tekst = 'thisisagenerator'
        sage: wektor = string_w_wektor_bajtow(tekst)
        sage: print(wektor)
            ['74','68','69','73','69','73','61','67','65','6e','65','72','61','74','6f','72']
    
    """
    
    return [hex(ord(i))[2:] for i in tekst]



def macierz_bajtow_w_string_bajtow(State):
    """
    Konwertuje macierz bajtów w napis bajtów (dwóch liczb szesnastkowych)
    
    
    WEJŚCIE:
        Macierz bajtów
        
    WYJŚCIE:
        Napis liczb szesnastkowych
        
    PRZYKŁAD:
        sage: State = [
                ['cc', 'a9', 'd7', '24'],
                ['9e', '31', '0b', 'a2'],
                ['a7', 'a6', 'd7', 'a5'],
                ['88', 'a9', '04', '3e']
            ]
        sage: napis = macierz_bajtow_w_string_bajtow(State)
        sage: print(napis)
            'CC A9 D7 24 9E 31 0B A2 A7 A6 D7 A5 88 A9 04 3E'
            
    """
    
    return "".join([element + " " for wektor in State for element in wektor])[:-1].upper()



def macierz_bajtow_w_string_znakow(State):
    """
    Konwertuje macierz bajtów w napis znaków
    
    
    WEJŚCIE:
        Macierz bajtów
        
    WYJŚCIE:
        Napis znaków
        
    PRZYKŁAD:
        sage: State = [
                ['49', '74', '27', '73'],
                ['20', '76', '69', '73'],
                ['69', '62', '6c', '65'],
                ['20', '74', '78', '74']
            ]
        sage: napis = macierz_bajtow_w_string_znakow(State)
        sage: print(napis)
            "It's visible txt"
            
    """
    
    return "".join([chr(Integer('0x'+element)) for wektor in State for element in wektor])



def bajty_w_macierz(State):
    """
    Konwertuje 16-bajtowy wektor, w macierz 4x4.
    
    
    WEJŚCIE:
        16-bajtowy wektor
        
    WYJŚCIE:
        Macierz 4x4
        
    PRZYKŁAD:
        sage: State = ['61', '70', '61', '73', '73', '77', '6F', '72', '64', '63', '69', '70', '74', '68', '65', '72']
        sage: State = bajty_w_macierz(State)
        sage: pp(State)
           [['61', '70', '61', '73'],
            ['73', '77', '6F', '72'],
            ['64', '63', '69', '70'],
            ['74', '68', '65', '72']]
            
    """
    
    return [list(State[i:i+4]) for i in range(0, len(State), 4)]



def bajty_w_liczby_dziesietne(State):
    """
    Konwertuje bajtową macierz w macierz liczb dziesiętnych.
    
    
    WEJŚCIE:
        Macierz bajtów w zapisie szesnastkowym
        
    WYJŚCIE:
        Dziesiętna macierz
        
    PRZYKŁAD:
        sage: State = [
                ['61', '70', '61', '73'],
                ['73', '77', '6F', '72'],
                ['64', '63', '69', '70'],
                ['74', '68', '65', '72']
            ]
        sage: State = bajty_w_liczby_dziesietne(State)
        sage: pp(State)
           [[ 97, 112,  97, 115],
            [115, 119, 111, 114],
            [100,  99, 105, 112],
            [116, 104, 101, 114]]
    
    """
    
    return map_threaded(lambda x: Integer('0x'+x), State)



def liczby_dziesietne_w_bajty(State):
    """
    Konwertuje dziesiętną macierz w macierz, której elementami są bajty.
    
    
    WEJŚCIE:
        Dziesiętna macierz
        
    WYJŚCIE:
        Macierz bajtów w zapisie szesnastkowym
        
    PRZYKŁAD:
        sage: State = [
                [ 97, 112,  97, 115],
                [115, 119, 111, 114],
                [100,  99, 105, 112],
                [116, 104, 101, 114]
            ]
        sage: State = liczby_dziesietne_w_bajty(State)
        sage: pp(State)
           [['61', '70', '61', '73'],
            ['73', '77', '6f', '72'],
            ['64', '63', '69', '70'],
            ['74', '68', '65', '72']]
    
    """
    
    return map_threaded(lambda x: hex(x).split('x')[-1].zfill(2), State)



def XOR_bajtow(a, b):
    """
    Zwraca wektor XOR'owanych wektorów a i b.


    WEJŚCIE:
        dwa wektory a i b
        
    WYJŚCIE:
        XOR'owany wektor
        
    PRZYKŁAD:
        sage: State = ['74', '68', '69', '73']
        sage: klucz = ['61', '70', '61', '73']
        sage: State = XOR_bajtow(State, klucz)
        sage: pp(State)
           ['15', '18', '08', '00']
           
    """
    
    return [hex(Integer('0x'+X)^^Integer('0x'+Y)).split('x')[-1].zfill(2) for (X,Y) in zip(a,b)]



def xtime(a):
    """
    Odpowiada za poprawne mnożenie w ciele GF(2^8).
    Sekcja 4.2.1 ze specyfikacji oraz sekcja 4.1.1 z 'The Design of Rijndael'
    
    """
    
    if a & 0x80:
        a = ((a << 1) ^^ 0x1B) & 0xFF
    else:
        a <<= 1

    return a

# Funkcja AddRoundKey

In [4]:
def AddRoundKey(State, RoundKey):
    """
    XOR'uje bajty tablicy stanu z bajtami podklucza
    (szczególnie w opisie funkcji XOR_bajtow).
    
    """
    
    return [XOR_bajtow(x,y) for (x,y) in zip(State, RoundKey)]


# Funkcja ByteSub

In [5]:
def ByteSub(State):
    """
    Przekształca tablicę stanu przez skrzynkę podstawieniową.
    
    
    REALIZACJA:
        Najpierw liczby w zapisie szesnastkowym są przekształcane 
        na liczby dziesiętne, zatem dla każdej liczby dziesiętnej 
        podstawiana jest liczba szesnastkowa, która znajduje
        się w macierzy S_box pod indeksem danej liczby dziesiętnej
        
    WEJŚCIE:
        Tablica stanu
        
    WYJŚCIE:
        Tablica stanu przekształcona przez S-box
        
    PRZYKŁAD:
        sage: State = [
                ['15', '18', '08', '00'],
                ['1a', '04', '0e', '15'],
                ['01', '0d', '0c', '02'],
                ['15', '1c', '0a', '00']
            ]
        sage: State = ByteSub(State)
        sage: pp(State)
           [['59', 'ad', '30', '63'],
            ['a2', 'f2', 'ab', '59'],
            ['7c', 'd7', 'fe', '77'],
            ['59', '9c', '67', '63']]
    
    """
    
    temp = bajty_w_liczby_dziesietne(State)
    return map_threaded(lambda i: S_box[i], temp)

# Odwrotna Funkcja InvByteSub

In [6]:
def InvByteSub(State):
    """
    Przekształca tablicę stanu przez odwrotną skrzynkę podstawieniową.
    
    
    REALIZACJA:
        Najpierw liczby w zapisie szesnastkowym są przekształcane 
        na liczby dziesiętne, zatem dla każdej liczby dziesiętnej 
        podstawiana jest liczba szesnastkowa, która znajduje
        się w macierzy Inv_S_box pod indeksem danej liczby dziesiętnej
        
    WEJŚCIE:
        Tablica stanu
        
    WYJŚCIE:
        Tablica stanu przekształcona przez odwrotną tablicę S-box
        
    PRZYKŁAD:
        sage: State = [
                ['59', 'ad', '30', '63'],
                ['a2', 'f2', 'ab', '59'],
                ['7c', 'd7', 'fe', '77'],
                ['59', '9c', '67', '63']
            ]
        sage: State = InvByteSub(State)
        sage: pp(State)
           [['15', '18', '08', '00'],
            ['1a', '04', '0e', '15'],
            ['01', '0d', '0c', '02'],
            ['15', '1c', '0a', '00']]
    
    """
    
    temp = bajty_w_liczby_dziesietne(State)
    temp = map_threaded(lambda i: Inv_S_box[i], temp)
    return transponowac(temp)

# Funkcja ShiftRows

In [7]:
def ShiftRows(State):
    """
    Przesuwa bajty w ostatnich trzech wierszach tablicy stanu.
    
    
    REALIZACJA:
        Najpierw dokonana jest transpozycja.
        Zatem przemieszczenie elementów tablicy dokonane jest za pomocą 'list slicing'
        na 1, 2, 3 bajty w lewo ze specyfikacji dla wierszy 2, 3, 4 odpowiednio.
        
    WEJŚCIE:
        Tablica stanu
        
    WYJŚCIE:
        Transponowana tablica stanu z przesuniętymi bajtami
        
    PRZYKŁAD:
        sage: State = [
                ['59', 'ad', '30', '63'],
                ['a2', 'f2', 'ab', '59'],
                ['7c', 'd7', 'fe', '77'],
                ['59', '9c', '67', '63']
            ]
        sage: State = ShiftRows(State)
        sage: pp(State)
           [['59', 'a2', '7c', '59'],
            ['f2', 'd7', '9c', 'ad'],
            ['fe', '67', '30', 'ab'],
            ['63', '63', '59', '77']]
        
    """
    
    State = transponowac(State)
    return [ 
             State[0],                    # Przepisujemy 1 wiersz
             State[1][1:]+State[1][:1],   # Przesuwamy drugi wiersz o 1 bajt w lewo
             State[2][2:]+State[2][:2],   # Przesuwamy trzeci wiersz o 2 bajty w lewo
             State[3][3:]+State[3][:3]    # Przesuwamy czwarty wiersz o 3 bajty w lewo
           ]

# Odwrotna Funkcja InvShiftRows

In [8]:
def InvShiftRows(State):
    """
    Przesuwa bajty w ostatnich trzech wierszach tablicy stanu.
    
    
    REALIZACJA:
        Najpierw dokonana jest transpozycja.
        Zatem przemieszczenie elementów tablicy dokonane jest za pomocą 'list slicing'
        na 1, 2, 3 bajty w prawo ze specyfikacji dla wierszy 2, 3, 4 odpowiednio.
        
    WEJŚCIE:
        Tablica stanu
        
    WYJŚCIE:
        Transponowana tablica stanu z przesuniętymi bajtami
        
    PRZYKŁAD:
        sage: State = [
                ['59', 'a2', '7c', '59'],
                ['f2', 'd7', '9c', 'ad'],
                ['fe', '67', '30', 'ab'],
                ['63', '63', '59', '77']
            ]
        sage: State = InvShiftRows(State)
        sage: pp(State)
           [['59', 'ad', '30', '63'],
            ['a2', 'f2', 'ab', '59'],
            ['7c', 'd7', 'fe', '77'],
            ['59', '9c', '67', '63']]
        
    """
    
    State = transponowac(State)
    return [ 
             State[0],                    # Przepisujemy 1 wiersz
             State[1][3:]+State[1][:3],   # Przesuwamy drugi wiersz o 1 bajt w prawo
             State[2][2:]+State[2][:2],   # Przesuwamy trzeci wiersz o 2 bajty w prawo
             State[3][1:]+State[3][:1]    # Przesuwamy czwarty wiersz o 3 bajty w prawo
           ]

# Funkcja MixColumns

In [9]:
def MixColumns(State):
    """
    Przemnaża bajty z kolumny tablicy stanu przez ustaloną macierz ze specyfikacji.
    
    
    REALIZACJA:
        Jest bardziej skomplikowana niż "proste" mnożenie.
        Mnożenie odbywa się w ciele GF(2^8).
        Realizowana za pomocą sekcji 4.2 ze specyfikacji oraz sekcji 4.1.2 z 'The Design of Rijndael'
    
    WEJŚCIE:
        Tablica stanu
        
    WYJŚCIE:
        Transponowana oraz przemnożona macierz
        
    PRZYKŁAD:
        sage: State = [
                ['59', 'a2', '7c', '59'],
                ['f2', 'd7', '9c', 'ad'],
                ['fe', '67', '30', 'ab'],
                ['63', '63', '59', '77']
            ]
        sage: State = MixColumns(State)
        sage: pp(State)
           [['22', 'dc', 'e9', '21'],
            ['39', 'dd', '1e', '8b'],
            ['2e', '56', '6b', '9a'],
            ['82', '89', '20', '03']]
    
    """
    
    S = bajty_w_liczby_dziesietne(State)
        
    for i in range(4):
        temp = S[0][i] ^^ S[1][i] ^^ S[2][i] ^^ S[3][i]
        pierwszy = S[0][i]
        
        S[0][i] ^^= temp ^^ xtime(S[0][i] ^^ S[1][i])
        S[1][i] ^^= temp ^^ xtime(S[1][i] ^^ S[2][i])
        S[2][i] ^^= temp ^^ xtime(S[2][i] ^^ S[3][i])
        S[3][i] ^^= temp ^^ xtime(S[3][i] ^^ pierwszy)

    return transponowac(liczby_dziesietne_w_bajty(S))

# Odwrotna Funkcja InvMixColumns

In [10]:
def InvMixColumns(State):
    """
    Przemnaża bajty z kolumny tablicy stanu przez ustaloną macierz ze specyfikacji.
    
    
    REALIZACJA:
        Jest bardziej skomplikowana niż "proste" mnożenie.
        Mnożenie odbywa się w ciele GF(2^8).
        Realizowana za pomocą sekcji 4.2 ze specyfikacji oraz sekcji 4.1.3 z 'The Design of Rijndael'
    
    WEJŚCIE:
        Tablica stanu
        
    WYJŚCIE:
        Transponowana oraz przemnożona macierz
        
    PRZYKŁAD:
        sage: State = [
                ['22', 'dc', 'e9', '21'],
                ['39', 'dd', '1e', '8b'],
                ['2e', '56', '6b', '9a'],
                ['82', '89', '20', '03']
            ]
        sage: State = InvMixColumns(State)
        sage: pp(State)
           [['59', 'a2', '7c', '59'],
            ['f2', 'd7', '9c', 'ad'],
            ['fe', '67', '30', 'ab'],
            ['63', '63', '59', '77']]
    
    """
    
    S = bajty_w_liczby_dziesietne(State)
        
    for i in range(4):
        u = xtime(xtime(S[i][0] ^^ S[i][2]))
        v = xtime(xtime(S[i][1] ^^ S[i][3]))
        
        S[i][0] ^^= u
        S[i][1] ^^= v
        S[i][2] ^^= u
        S[i][3] ^^= v
        
    S = transponowac(liczby_dziesietne_w_bajty(S))

    return MixColumns(S)

# Funkcja AES_block_encrypt

In [11]:
def AES_block_encrypt(tekst, klucz_glowny):
    """
    Funkcja AES_block_encrypt najpierw dokonuje przetwarzania danych, zatem szyfruje blok tekstu jawnego podanym kluczem.
    Aby ułatwić napisanie kodu (indeksowanie po macierzy) w większości funkcji działania są prowadzone na transponowanym bloku.
    
    WEJŚCIE:
        tekst: tekst jawny w postaci 16-znakowego napisu
        klucz_glowny: 16, 24, 32-znakowy klucz
        
    WYJŚCIE:
        Zaszyfrowany tekst w postaci bajtów oddzielonych spacją, zapisanych jako dwa znaki szesnastkowe 
        
    PRZYKŁAD:
        sage: tekst = 'It's visible txt'
        sage: klucz_glowny = 'Super Cipher Key'
        sage: szyfrogram = AES_block_encrypt(tekst, klucz_glowny)
        sage: print(szyfrogram)
            '5e fd c0 ee 0e 0d 2c b0 ba 80 3d fb be c8 70 e7'
    
    """
    
    ##################################
    ##### Wstępna obróbka danych #####
    ##################################
    
    # Przekształcamy string w wektor, którego elementami są bajty, które składają się z dwóch liczb szesnastkowych
    tekst = string_w_wektor_bajtow(tekst)
    klucz_glowny = string_w_wektor_bajtow(klucz_glowny)
    
    # Wyliczamy liczbę rund w zależności od rozmiaru klucza
    Nr = len(klucz_glowny) // 4 + 6
    
    # Konwertujemy macierz bajtów tekstu jawnego w macierz 4x4
    State = bajty_w_macierz(tekst)

    # Tworzymy macierz kluczy dla wszystkich rund
    macierz_kluczy = KeyExpansion(klucz_glowny)


    ################################
    ##### Algorytm szyfrowania #####
    ################################
    
    # Runda zerowa:
    State = AddRoundKey(State, macierz_kluczy[0])
    
    # Runda podstawowa:
    for i in range(1,Nr):
        State = ByteSub(State)
        State = ShiftRows(State)
        State = MixColumns(State)
        State = AddRoundKey(State, macierz_kluczy[i])
        
    # Runda końcowa:
    State = ByteSub(State)
    State = ShiftRows(State)
    State = AddRoundKey(transponowac(State), macierz_kluczy[Nr])
    
    return macierz_bajtow_w_string_bajtow(State)

# Funkcja AES_block_decrypt

In [12]:
def AES_block_decrypt(szyfrogram, klucz_glowny):
    """
    Funkcja AES_block_decrypt najpierw dokonuje przetwarzania danych, zatem deszyfruje blok szyfrogramu podanym kluczem.
    Aby ułatwić napisanie kodu (indeksowanie po macierzy) w większości funkcji działania są prowadzone na transponowanym bloku
    
    WEJŚCIE:
        szyfrogram: tekst zaszyfrowany w postaci bajtów oddzielonych spacją, zapisanych jako dwa znaki szesnastkowe
        klucz_glowny: 16, 24, 32-znakowy klucz
        
    WYJŚCIE:
        Tekst jawny
        
    PRZYKŁAD:
        sage: szyfrogram = '5e fd c0 ee 0e 0d 2c b0 ba 80 3d fb be c8 70 e7'
        sage: klucz_glowny = 'Super Cipher Key'
        sage: tekst = AES_block_decrypt(szyfrogram, klucz_glowny)
        sage: print(tekst)
            'It's visible txt'
    
    """
    
    ##################################
    ##### Wstępna obróbka danych #####
    ##################################
    
    # Przekształcamy string w wektor, którego elementami są bajty, które składają się z dwóch liczb szesnastkowych
    szyfrogram = szyfrogram.split(" ")
    klucz_glowny = string_w_wektor_bajtow(klucz_glowny)
    
    # Wyliczamy liczbę rund w zależności od rozmiaru klucza
    Nr = len(klucz_glowny) // 4 + 6
    
    # Konwertujemy macierz bajtów szyfrogramu w macierz 4x4
    State = bajty_w_macierz(szyfrogram)
    
    # Tworzymy macierz kluczy dla wszystkich rund
    macierz_kluczy = KeyExpansion(klucz_glowny)


    ##################################
    ##### Algorytm deszyfrowania #####
    ##################################
    
    # Odwrotna runda zerowa:
    State = AddRoundKey(State, macierz_kluczy[-1])
    State = InvShiftRows(State)
    State = InvByteSub(State)
    
    # Odwrotna runda podstawowa:
    for i in range(Nr - 1, 0, -1):
        State = AddRoundKey(State, macierz_kluczy[i])
        State = InvMixColumns(State)        
        State = InvShiftRows(State)    
        State = InvByteSub(State)

    # Odwrotna runda końcowa:
    State = AddRoundKey(State, macierz_kluczy[0])
    
    return macierz_bajtow_w_string_znakow(State)

# Algorytm generowania podkluczy KeyExpansion

In [13]:
def KeyExpansion(klucz_glowny):
    """
    Funkcja KeyExpansion tworzy macierz podkluczy dla wszystkich rund, także każdy klucz jest macierzą.
    
    
    WEJŚCIE:
        klucz_glowny: 16, 24, 32-elementowa macierz, której elementami są bajty, zapisane w postaci dwóch liczb szesnastkowych
        
    WYJŚCIE:
        Macierz podkluczy
        
    PRZYKŁAD:
        sage: klucz_glowny = ['53','75','70','65','72','20','43','69','70','68','65','72','20','4b','65','79']
        sage: macierz_kluczy = KeyExpansion(klucz_glowny)
        sage: pp(macierz_kluczy)
            [[['53', '75', '70', '65'],
              ['72', '20', '43', '69'],
              ['70', '68', '65', '72'],
              ['20', '4b', '65', '79']],
             [['e1', '38', 'c6', 'd2'],
              ['93', '18', '85', 'bb'],
              ['e3', '70', 'e0', 'c9'],
              ['c3', '3b', '85', 'b0']],
              ...
            ]
    
    """
    
    # Inicjalizujemy macierz kluczy dla rund pierwszym głównym kluczem
    wektory_kluczy = bajty_w_macierz(klucz_glowny)
    
    # Wyliczamy współczynniki dla algorytmu
    Nk = len(klucz_glowny) // 4
    Nr = Nk + 6
    Nb = 4
    
    i = 1
    while len(wektory_kluczy) < (Nr + 1) * Nb:
        
        # Kopiujemy poprzednie słowo (wektor)
        slowo = list(wektory_kluczy[-1])

        # Kolejno tworzymy wektor podlucza
        if len(wektory_kluczy) % Nk == 0:

            # RotWord - przesunięcie bajtów w lewo
            slowo.append(slowo.pop(0))
            
            # SubWord - przepuszczenie prez S-skrzynkę
            slowo = ByteSub(slowo)

            # XOR'owanie pierwszego bajtu słowa z bajtem tablicy R_con, ponieważ inne bajty tablicy mają wartość 0
            slowo[0] = hex(Integer('0x'+slowo[0])^^R_con[i]).split('x')[-1].zfill(2)
            i += 1
            
        elif len(klucz_glowny) == 32 and len(wektory_kluczy) % Nk == 4:
            # Przepuść słowo przez S-skrzynkę dla czwartej iteracji, jeśli wykorzystany 256-bitowy klucz
            slowo = ByteSub(slowo)

        # XOR'owanie z równoważnym słowem z poprzedniej iteracji
        slowo = XOR_bajtow(slowo, wektory_kluczy[-Nk])
        wektory_kluczy.append(slowo)

    # Grupowanie słów (wektorów) kluczy w macierze 4x4
    return [wektory_kluczy[4*i : 4*(i+1)] for i in range(len(wektory_kluczy) // 4)]

# Główna Funkcja sterująca działaniem algorytmu

In [14]:
def main():
    while True:
        print('=' * 80)
        tryb = input("Wybierz tryb działania:\n'0' - skończyć działanie, '1' - szyfrowanie, '2' - deszyfrowanie,: ")
        print()

        if tryb not in ['0', '1', '2']:
            print()
            print(f"Podano niepoprawny tryb pracy '{tryb}', musi być '0', '1' albo '2'\nSpóbuj ponownie.")
            continue
        
        if tryb == '0':
            print('Skończono')
            return
        elif tryb == '1':
            wynik = encrypt()
        elif tryb == '2':
            wynik = decrypt()
        
        print()
        print("Wynik końcowy:")
        print(wynik)

        
def encrypt():
    
    while True:
        tekst_jawny = input("Wpisz 16-znakowy tekst jawny: ")
        if czy_poprawny_tekst(tekst_jawny):
            break
    
    print()
    
    while True:
        klucz_glowny = input("Wpisz 16, 24 albo 32-znakowy klucz: ")
        if czy_poprawny_klucz(klucz_glowny):
            break

    return AES_block_encrypt(tekst_jawny, klucz_glowny)
    
        
def decrypt():
    
    while True:
        szyfrogram = input("Wpisz 16-znakowy tekst zaszyfrowany w takiej postaci, jak wynik końcowy po szyfrowaniu.\n" \
                           "Każdy znak jest oddzielony spacją oraz jest zapisany jako dwie liczby szesnastkowe:\n" \
                           "(np. '5e fd c0 ee 0e 0d 2c b0 ba 80 3d fb be c8 70 e7')\n")
        if czy_poprawny_tekst(szyfrogram.split(" ")):
            break
    
    print()
    
    while True:
        klucz_glowny = input("Wpisz 16, 24 albo 32-znakowy klucz: ")
        if czy_poprawny_klucz(klucz_glowny):
            break

    return AES_block_decrypt(szyfrogram, klucz_glowny)

    
def czy_poprawny_tekst(tekst):

    if len(tekst) != 16:
        print()
        print(f"Niepoprawna długość tekstu: {len(tekst)}, musi być 16 znaków\nSpóbuj ponownie.")
        return False
    
    return True
    

def czy_poprawny_klucz(klucz):

    if len(klucz) not in [16,24,32]:
        print()
        print(f"Niepoprawna długość klucza: {len(klucz)}, musi być 16, 24 albo 32 znaków\nSpóbuj ponownie.")
        return False
    
    return True

In [ ]:
main()